# Working with Trajectories

In this example you will see QMzyme can be used to generate multiple QMzyme Models from different trajectory snapshots. 

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [XTB_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#xtb-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [5]:
import QMzyme
from QMzyme.SelectionSchemes import DistanceCutoff
from QMzyme.data import PQR, DCD

In [2]:
# How many frames?
model = QMzyme.GenerateModel(PQR, DCD)
print("Number of Frames: ", model.universe.trajectory.n_frames)

Number of Frames:  50


/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/hrk/git/QMzyme/QMzyme/MDAnalysisWrapper.py:28: UserWarning: Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.
  warnings.warn("Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.", UserWarning)


#### QM-only: Loop over frames to generate QMzymeModels
In this first scenario, you will generate five QM-only models using the DistanceCutoff selection scheme with a cutoff of 3 Angstroms.

In [3]:
%%time

qm_method = QMzyme.QM_Method(
    basis_set='6-31G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = QMzyme.GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(selection=DistanceCutoff, cutoff=3)
    m.cutoff_3.rename(f'{m.cutoff_3.name}_frame_{frame}')
    c_alpha_atoms = m.regions[-1].get_atoms(attribute='name', value='CA')
    m.regions[-1].set_fixed_atoms(atoms=c_alpha_atoms)
    qm_method.assign_to_region(region=m.regions[-1])
    m.truncate()
    m.write_input(filename=f"{m.name}_cutoff3_frame{frame}_qm")


             Frame 0

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QM. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff3_frame0_qm.inp created.

             Frame 10

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QM. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff3_frame10_qm.inp created.

             Frame 20

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QM. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff3_frame20_qm.inp created.

             Frame 30

Truncated model has been created and saved to attribute 'truncated' and stored in QM

#### QM/xTB: Loop over frames to generate QMzymeModels
In this second scenario, you will generate five QM/xTB models with a QM region consisting of the ligand, EQU and the catalytic ASP103 residue and a large xTB region containing residues within 8 Angstroms of EQU.

In [4]:
%%time

qm_method = QMzyme.QM_Method(
    basis_set='6-31+G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

xtb_method = QMzyme.XTB_Method()

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = QMzyme.GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(name='qm_region', selection='resid 263 or resid 103')
    m.set_region(selection=DistanceCutoff, cutoff=8)
    m.cutoff_8.rename(f'{m.cutoff_8.name}_frame_{frame}')
    qm_method.assign_to_region(region=m.qm_region)
    xtb_method.assign_to_region(region=m.cutoff_8)
    m.truncate()
    m.write_input(filename=f"{m.name}_cutoff8_frame{frame}_qmxtb")


             Frame 0

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QMXTB. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff8_frame0_qmxtb.inp created.

             Frame 10

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QMXTB. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff8_frame10_qmxtb.inp created.

             Frame 20

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QMXTB. This model will be used to write the calculation input.
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/1oh0_equ_cutoff8_frame20_qmxtb.inp created.

             Frame 30

Truncated model has been created and saved to attribute 'truncated